In [2]:
from src.models import Sequential
from src.layers import *
from src.activation_functions import *
from src.loss_functions import *

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [3]:
the_image = np.random.random((1, 1, 28, 28))

In [4]:
conv = Conv2D(1, 3, 1, 1)

In [5]:
conv.forward(the_image).shape

(1, 1, 28, 28)

In [6]:
the_error = np.load('error.npy')

In [7]:
new_error = np.expand_dims(the_error[0], 0)
new_error.shape

(1, 1, 28, 28)

In [8]:
weig = conv.weights

In [9]:
weig.shape

(1, 3, 3)

In [10]:
new_error_padded = np.pad(new_error, ((0, 0), (0, 0), (1, 1), (1, 1)))
new_error_padded.shape

(1, 1, 30, 30)

In [11]:
conv.backward(new_error, 1)

array([[[[6.54264720e-312, 6.54264777e-312, 2.21510998e+214,
          8.88782353e+252, 7.98934359e+289, 3.12121805e+209,
          1.16979977e+171, 6.01334688e-154, 1.03627183e-013,
          1.03878562e-013, 1.03877586e-013, 1.03878562e-013,
          1.03878562e-013, 1.03878562e-013, 4.83865671e-014,
          1.03878562e-013, 1.03878562e-013, 6.01346953e-154,
          6.01399352e-154, 6.02406676e-154, 6.01347206e-154,
          6.01347040e-154, 6.01347002e-154, 6.01356160e-154,
          3.06489452e-086, 6.01347002e-154, 6.01347002e-154,
          6.01347002e-154],
         [4.47593814e-091, 4.47593816e-091, 4.47593816e-091,
          4.47593816e-091, 4.47593816e-091, 4.47593816e-091,
          4.47593348e-091, 4.47593816e-091, 4.47593816e-091,
          6.01347002e-154, 3.12237215e-120, 6.01347018e-154,
          6.96312257e-077, 7.85739828e+289, 6.96312257e-077,
          6.01347002e-154, 1.47058722e+290, 1.95994476e-062,
          7.85739828e+289, 6.01347002e-154, 4.47593816e-0

In [12]:
conv.weights

array([[[-0.6381846 , -0.7707853 ,  0.13512868],
        [-0.85111016, -0.5180926 , -0.6469569 ],
        [-0.01240616, -0.12029199, -0.9638308 ]]], dtype=float32)

In [99]:
test_error = np.asarray([[
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]])
print(test_error.shape)

test_error_padded = np.pad(test_error[0], 1)
test_error2 = np.expand_dims(test_error_padded, (0, 1))

(1, 6, 5)


In [100]:
test_error2.shape

(1, 1, 8, 7)

In [95]:
test_error

array([[1],
       [1]])

In [16]:
weig = np.asarray([[
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
],
[
    [1, 0, -1],
    [2, 0, -2],
    [1, 0, -1]
]])

In [17]:
weig.shape

(2, 3, 3)

In [102]:
error_conv = np.zeros((1, 1, 6, 5))
error_conv.shape

(1, 1, 6, 5)

In [103]:
for b in range(error_conv.shape[0]):
    for f in range(error_conv.shape[1]):
        for h in range(error_conv.shape[2]):
            for w in range(error_conv.shape[3]):
                error_conv[b, f, h, w] = np.sum(
                    test_error2[b, f, h:h+3, w:w+3] * weig[f]
                )

In [104]:
error_conv.shape

(1, 1, 6, 5)

In [105]:
error_conv[0]

array([[[ 1.,  1.,  0., -1., -1.],
        [ 3.,  3.,  0., -3., -3.],
        [ 4.,  4.,  0., -4., -4.],
        [ 4.,  4.,  0., -4., -4.],
        [ 3.,  3.,  0., -3., -3.],
        [ 1.,  1.,  0., -1., -1.]]])

In [22]:
import scipy

In [23]:
scipy.signal.correlate2d(test_error[0][0], weig[0])

ValueError: correlate2d inputs must both be 2-D arrays

In [ ]:
test_error[0][0][1:-1, 1:-1] * error_conv[0][0]

In [115]:
test_error2 = np.asarray([[
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]])

test_error = test_error[1:-1, 1:-1]

test_inp = np.asarray([[
    [1, 1, 1, 2, 3],
    [1, 1, 1, 2, 3],
    [1, 1, 1, 2, 3],
    [2, 2, 2, 2, 3],
    [3, 3, 3, 3, 3],
    [4, 4, 4, 4, 4],
], [
    [1, 1, 1, 2, 3],
    [1, 1, 1, 2, 3],
    [1, 1, 1, 2, 3],
    [2, 2, 2, 2, 3],
    [3, 3, 3, 3, 3],
    [4, 4, 4, 4, 4],
]])

test_weig = np.asarray([[
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
],[
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
]])

In [116]:
test_grad = np.zeros((2, 3, 3))

In [117]:
# for b in range(test_grad.shape[0]):
for h in range(test_grad.shape[1]):
    for w in range(test_grad.shape[2]):
        test_grad[:, h, w] = np.sum(
            test_inp[:, h:h+test_error.shape[0], w:w+test_error.shape[1]] * test_error
        , axis=(1, 2))

ValueError: operands could not be broadcast together with shapes (2,0,4) (0,4,5) 

In [118]:
print(test_grad)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [119]:
test_error2 = np.expand_dims(test_error2, 0)

In [120]:
test_error2.shape

(1, 1, 6, 5)

In [121]:
test_error2

array([[[[0, 0, 0, 0, 0],
         [0, 1, 1, 1, 0],
         [0, 1, 1, 1, 0],
         [0, 1, 1, 1, 0],
         [0, 1, 1, 1, 0],
         [0, 0, 0, 0, 0]]]])

In [122]:
test_im_err = np.zeros((1, 1, 6, 5))

In [123]:
test_weig.shape

(2, 3, 3)

In [127]:
# for b in range(test_grad.shape[0]):
for h in range(6):
    for w in range(5):
        test_im_err[:, :, h, w] = np.sum(
            test_error2[:, :, h:h+3, w:w+3] * test_weig)

ValueError: operands could not be broadcast together with shapes (1,1,3,2) (2,3,3) 

In [114]:
test_im_err

array([[[[ 1.,  1.,  0., -1., -1.],
         [ 3.,  3.,  0., -3., -3.],
         [ 4.,  4.,  0., -4., -4.],
         [ 4.,  4.,  0., -4., -4.],
         [ 3.,  3.,  0., -3., -3.],
         [ 1.,  1.,  0., -1., -1.]]]])

In [109]:
scipy.signal.correlate2d(test_error2[0][0], test_weig[0])

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  1,  0, -1, -1,  0,  0],
       [ 0,  0,  3,  3,  0, -3, -3,  0,  0],
       [ 0,  0,  4,  4,  0, -4, -4,  0,  0],
       [ 0,  0,  4,  4,  0, -4, -4,  0,  0],
       [ 0,  0,  3,  3,  0, -3, -3,  0,  0],
       [ 0,  0,  1,  1,  0, -1, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]])